In [1]:
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
#import tensorflow as tf


# porting part from keras to PyTorch
# keras
#from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
#from keras.layers import Activation, Dense, Dropout, Flatten
#from keras.layers.normalization import BatchNormalization
#from keras.models import Sequential
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# PyTorch
import torch
import torch.nn

In [38]:
# Get images
#image = img_to_array(load_img('woman.jpg'))
#image = np.array(image, dtype=float)

#image = torch.utils.data.DataLoader(dataset="woman.jpg")

#Deep learning pdf attempt
import imageio
img_arr = imageio.imread('woman.jpg')
img_arr.shape #(400,400, 4) 3 dimensions, 2 spacial (width&height), 1 color channel (RGB)
#type(img_arr)

(400, 400, 4)

In [39]:
#convert numpy-like array into Tensor
#img2 = torch.Tensor(img_arr) - instead of 255 it's 255.
img = torch.from_numpy(img_arr)

In [29]:
#now, change the layout of the Tensor by Transposing it
#PyTorch modules that work with images need them to be in C x H x W order (channel, height, width)
#so far the image is in W x H x C layout
#let's swap the first and last channel.
out = torch.transpose(img, 0, 2)
#changing img, changes out!!!

#to create a data set of multiple images to use as an input for your neural networks,
#you store the images in a batch along the first dimension to obtain aN x C x H x W tensor

In [30]:
# A more efficient way to using stack to build up a tensor is to preallocate a tensor of appropriate size
# and fill it with images loaded from a directory.
#This indicates that the batch consists of 100 pictures, 256x256 in size.
#Dtype is set to int8 as that's the standard value in most cameras.


batch_size = 100
batch = torch.zeros(100, 3, 256, 256, dtype=torch.uint8)

In [34]:
#import os
#os.getcwd() - get current working directory

In [35]:
#Loading all images from a directory and saving them in a tensor.
import os
data_dir = "/home/hyperlink/Desktop/KeepingUpAppearances-master/"

#get all .jpg items in this dir
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name) == ".jpg"]

for i, filename in enumerate(filenames):
    img_arr = imageio.imread(filename)
    #correct the images to the correct layer order.
    batch[i] = torch.transpose(torch.from_numpy(img_arr), 0, 2)


In [41]:
#Neural Networks have the best performance with input data values between (0 and 1) or (-1 and 1)
#They also usually work with floating-point tensors as their input.
#A typical think you might need to do is cast the tensor to float, which is easy.
batch = batch.float()
#Second thing you might want to do is to normalise the data (flatten it to values between 0s and 1s)
#This is a bit trickier, but one of the things people can do is divide the number by 255.
#255 is the maximum representable number in 8-bit unsigned int
batch /= 255.0

#ALTERNATIVE
#You could also compute the mean and standard deviation of the input data and scale it.
#Make it so that the output has zero mean and and unit standard deviation across each channel.
#Produces a RuntimeError that it got Byte instead of floating type in mean = torch. line
#n_channels = batch.shape[1]
#for c in range(n_channels):
#     mean = torch.mean(batch[:, c])
#     std = torch.std(batch[:, c])
#     batch[:, c] = (batch[:, c]-mean) / std

#You can perform several other operations on inputs.
#One of those is called geometric transformations, such as rotation, scaling, and cropping.
#This operations might help with training.